# Generating Word Embeddings - Lab

## Introduction

In this lab, you'll learn how to generate word embeddings by training a Word2Vec model, and then embedding layers into Deep Neural Networks for NLP!

## Objectives

You will be able to:

* Demonstrate a basic understanding of the architecture of the Word2Vec model
* Demonstrate an understanding of the various tunable parameters of word2vec such as vector size and window size

## Getting Started

In this lab, you'll start by creating your own word embeddings by making use of the Word2Vec Model. Then, you'll move onto building Neural Networks that make use of **_Embedding Layers_** to accomplish the same end-goal, but directly in your model. 

As you've seen, the easiest way to make use of Word2Vec is to import it from the [Gensim Library](https://radimrehurek.com/gensim/). This model contains a full implementation of Word2Vec, which you can use to begin training immediately. For this lab, you'll be working with the [News Category Dataset from Kaggle](https://www.kaggle.com/rmisra/news-category-dataset/version/2#_=_).  This dataset contains headlines and article descriptions from the news, as well as categories for which type of article they belong to.

Run the cell below to import everything you'll need for this lab. 

In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)
from gensim.models import Word2Vec
from nltk import word_tokenize

/opt/conda/envs/learn-env/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


Now, import the data. The data stored in the file `'News_Category_Dataset_v2.json'`.  This file is compressed, so that it can be more easily stored in a github repo. **_Make sure to unzip the file before continuing!_**

In the cell below, use the `read_json` function from pandas to read the dataset into a DataFrame. Be sure to include the parameter `lines=True` when reading in the dataset!

Once you've loaded in the data, inspect the head of the DataFrame to see what your data looks like. 

In [2]:
import zipfile
zip_ref = zipfile.ZipFile('News_Category_Dataset_v2.zip', 'r')
zip_ref.extractall('.')
zip_ref.close()

In [3]:
!ls

CONTRIBUTING.md  LICENSE.md			News_Category_Dataset_v2.zip
index.ipynb	 News_Category_Dataset_v2.json	README.md


In [4]:
raw_df = pd.read_json('News_Category_Dataset_v2.json', lines=True)
raw_df.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


## Preparing the Data

Since you're working with text data, you need to do some basic preprocessing including tokenization. Notice from the data sample that two different columns contain text data--`headline` and `short_description`. The more text data your Word2Vec model has, the better it will perform. Therefore, you'll want to combine the two columns before tokenizing each comment and training your Word2Vec model. 

In the cell below:

* Create a column called `combined_text` that consists of the data from `df.headline` plus a space character (`' '`) plus the data from `df.short_description`.
* Use the `combined_text` column's `map()` function and pass in `word_tokenize`. Store the result returned in `data`.

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/casual-
[nltk_data]     port-1504/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
raw_df['combined_text'] = raw_df.headline + ' ' + raw_df.short_description
data = raw_df.combined_text.map(word_tokenize)

Inspect the first 5 items in `data` to see how everything looks. 

In [7]:
data[:5]

0    [There, Were, 2, Mass, Shootings, In, Texas, L...
1    [Will, Smith, Joins, Diplo, And, Nicky, Jam, F...
2    [Hugh, Grant, Marries, For, The, First, Time, ...
3    [Jim, Carrey, Blasts, 'Castrato, ', Adam, Schi...
4    [Julianna, Margulies, Uses, Donald, Trump, Poo...
Name: combined_text, dtype: object

Notice that although the words are tokenized, they are still in the same order they were in as headlines. This is important, because the words need to be in their original order for Word2Vec to establish the meaning of them. Remember that for a Word2Vec model you can specify a  **_Window Size_** that tells the model how many words to take into consideration at one time. 

If your window size was 5, then the model would start by looking at the words "Will Smith joins Diplo and", and then slide the window by one, so that it's looking at "Smith joins Diplo and Nicky", and so on, until it had completely processed the text example at index 1 above. By doing this for every piece of text in the entire dataset, the Word2Vec model learns excellent vector representations for each word in an **_Embedding Space_**, where the relationships between vectors capture semantic meaning (recall the vector that captures gender in the previous "king - man + woman = queen" example you saw).

Now that you've prepared the data, train your model and explore a bit!

## Training the Model

Start by instantiating a Word2Vec Model from gensim below. 

In the cell below:

* Create a `Word2Vec` model and pass in the following arguments:
    * The dataset we'll be training on, `data`
    * The size of the word vectors to create, `size=100`
    * The window size, `window=5`
    * The minimum number of times a word needs to appear in order to be counted in  the model, `min_count=1`.
    * The number of threads to use during training, `workers=4`

In [8]:
model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

Now, that you've instantiated Word2Vec model, train it on your text data. 

In the cell below:

* Call `model.train()` and pass in the following parameters:
    * The dataset we'll be training on, `data`
    * The `total_examples`  of sentences in the dataset, which you can find in `model.corpus_count`. 
    * The number of `epochs` you want to train for, which we'll set to `10`

In [9]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(55563493, 67352790)

Great! you now have a fully trained model! The word vectors themselves are stored inside of a `Word2VecKeyedVectors` instance, which is stored inside of `model.wv`. To simplify this, restore this object inside of the variable `wv` to save yourself some keystrokes down the line. 

In [10]:
wv = model.wv

## Examining Your Word Vectors

Now that you have a trained Word2Vec model, go ahead and explore the relationships between some of the words in the corpus! 

One cool thing you can use Word2Vec for is to get the most similar words to a given word. You can do this passing in the word to `wv.most_similar()`. 

In the cell below, try getting the most similar word to `'Texas'`.

In [11]:
wv.most_similar('Texas')

[('Oklahoma', 0.8000996112823486),
 ('Illinois', 0.79960036277771),
 ('Massachusetts', 0.7858473062515259),
 ('Oregon', 0.7851098775863647),
 ('Ohio', 0.7842681407928467),
 ('Pennsylvania', 0.783302366733551),
 ('Connecticut', 0.7830711603164673),
 ('Florida', 0.7802234888076782),
 ('Louisiana', 0.7788400650024414),
 ('Georgia', 0.7730224132537842)]

Interesting! All of the most similar words are also states. 

You can also get the least similar vectors to a given word by passing in the word to the `most_similar()` function's `negative` parameter. 

In the cell below, get the least similar words to `'Texas'`.

In [12]:
wv.most_similar(negative=['Texas'])

[('On-the-go', 0.4393371343612671),
 ('sparks', 0.41945552825927734),
 ('SHUCK', 0.393781840801239),
 ('self-thoughts', 0.39284858107566833),
 ('TV/radio', 0.382450133562088),
 ('table-approved', 0.37995776534080505),
 ('moaned', 0.37759530544281006),
 ('savor', 0.37391364574432373),
 ('bundt', 0.3693310618400574),
 ('forethoughts', 0.36852556467056274)]

This seems like random noise. It is a result of the way Word2Vec is computing the similarity between word vectors in the embedding space. Although the word vectors closest to a given word vector are almost certainly going to have similar meaning or connotation with your given word, the word vectors that the model considers 'least similar' are just the word vectors that are farthest away, or have the lowest cosine similarity. It's important to understand that while the closest vectors in the embedding space will almost certainly share some level of semantic meaning with a given word, there is no guarantee that this relationship will hold at large distances. 

You can also get the vector for a given word by passing in the word as if you were passing in a key to a dictionary. 

In the cell below, get the word vector for `'Texas'`.

In [13]:
wv['Texas']

array([-1.4181209e-01, -7.5965315e-01, -1.6511123e+00,  4.5613131e-01,
        1.3577507e+00, -3.4955745e+00, -1.8432492e+00, -8.8069129e-01,
       -3.2011056e+00, -1.8014889e+00,  1.0293828e+00, -3.6128357e-01,
        3.6502984e+00,  1.6419925e-01,  1.0397955e+00,  6.3441640e-01,
       -3.8786814e-01, -1.1152669e+00,  1.9935585e+00, -1.0800232e+00,
        1.1411180e+00, -9.7375266e-02, -1.1708570e+00,  9.8519310e-02,
       -7.4144833e-02, -8.3182573e-01,  9.4744438e-01, -9.5965952e-01,
       -8.5536778e-01,  1.4600797e+00, -1.3591406e+00, -1.8338223e+00,
        7.0493639e-01, -1.5735635e+00, -1.1393286e+00,  3.1263587e-01,
        1.0661962e+00, -3.1875849e+00,  2.8245378e-02, -1.5385946e+00,
       -7.2130966e-01, -3.7349302e-01,  1.4813050e+00, -1.6526777e+00,
       -4.7162408e-01, -2.1204343e+00,  4.3555379e-01,  1.2409644e+00,
        7.1478254e-01, -4.7707251e-01, -1.5034366e+00,  9.2349970e-01,
        9.8946415e-02,  1.3788413e+00, -6.1964754e-02,  5.5366224e-01,
      

Now get all of the word vectors from the object at once. You can find these inside of `wv.vectors`. Try it out in the cell below.  

In [14]:
wv.vectors

array([[-1.1284779e+00, -1.2070370e+00,  7.8556955e-01, ...,
        -1.1570977e+00,  1.1671801e+00,  6.6094983e-01],
       [-1.3014082e+00,  1.0846396e-01, -7.0508486e-01, ...,
        -1.4710283e+00,  4.1425011e-01, -3.6597955e-01],
       [-1.4522485e-02,  9.7458482e-01, -7.5188184e-01, ...,
        -2.0712810e+00, -1.1298232e+00,  3.7414244e-01],
       ...,
       [-2.9998295e-02,  2.1008966e-03, -2.9473914e-02, ...,
         9.9934660e-02, -8.7298624e-02, -2.1989081e-02],
       [-2.4010938e-02,  5.1286560e-02, -5.2008945e-03, ...,
        -1.0196371e-02,  6.1778212e-03, -8.3274944e-03],
       [ 1.9896347e-03, -1.8433705e-02, -3.7262250e-02, ...,
         7.9609886e-02,  1.2561339e-02, -1.2107197e-02]], dtype=float32)

As a final exercise, try to recreate the _'king' - 'man' + 'woman' = 'queen'_ example previously mentioned. You can do this by using the `most_similar` function and translating the word analogies into an addition/subtraction formulation (as shown above). Pass the original comparison, which you are calculating a difference between, to the negative parameter, and the analogous starter you want to apply the same transformation to, to the `positive` parameter.

Do this now in the cell below. 

In [15]:
wv.most_similar(positive=['King', 'Woman'], negative=['man'])

[('queen', 0.6367260217666626),
 ('revival', 0.6041049957275391),
 ('reminiscent', 0.6028562784194946),
 ('magic', 0.593749463558197),
 ('symbol', 0.5867269039154053),
 ('title', 0.583415150642395),
 ('jewel', 0.5762290954589844),
 ('brunette', 0.5713697671890259),
 ('novel', 0.5699683427810669),
 ('crown', 0.5637811422348022)]

As you can see from the output above, your model isn't perfect, but 'Queen' is still in the top 3, and with 'Princess' not too far behind. As you can see from the word in first place, 'reminiscent', your model is far from perfect. This is likely because you didn't have enough training data. That said, given the small amount of training data provided, the model still performs remarkably well! 

In the next lab, you'll reinvestigate transfer learning, loading in the weights from an open-sourced model that has already been trained for a very long time on a massive amount of data. Specifically, you'll work with the GloVe model from the Stanford NLP Group. There's not really any benefit from training the model ourselves, unless your text uses different, specialized vocabulary that isn't likely to be well represented inside an open-source model.

## Summary

In this lab, you learned how to train and use a Word2Vec model to created vectorized word embeddings!